In [1]:
import os
import onnx
import paddle
import numpy as np
import pandas as pd
import onnxruntime as ort
import paddle.nn.functional as F
from paddle.metric import Accuracy
from paddle.static import InputSpec
from sklearn.metrics import accuracy_score

# Param Config

In [2]:
N_EPOCH = 2
N_BATCH = 64
N_BATCH_NUM = 250
S_DATA_PATH = r"mnist_train.csv"
S_PADDLE_MODEL_PATH = r"cnn_model"
S_ONNX_MODEL_PATH = r"cnn_model_batch%d.onnx" % N_BATCH
S_DEVICE, N_DEVICE_ID, S_DEVICE_FULL = "gpu", 0, "gpu:0"
# S_DEVICE, N_DEVICE_ID, S_DEVICE_FULL = "cpu", 0, "cpu"
paddle.set_device(S_DEVICE_FULL)

CUDAPlace(0)

# Read Data

In [3]:
df = pd.read_csv(S_DATA_PATH, header=None)
print(df.shape)
np_mat = np.array(df)
print(np_mat.shape)

X = np_mat[:, 1:]
Y = np_mat[:, 0]
X = X.astype(np.float32) / 255

X_train = X[:N_BATCH * N_BATCH_NUM]
X_test = X[N_BATCH * N_BATCH_NUM:]
Y_train = Y[:N_BATCH * N_BATCH_NUM]
Y_test = Y[N_BATCH * N_BATCH_NUM:]
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28)
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28)
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

class MnistDataSet(paddle.io.Dataset):
    def __init__(self, X, Y):
        self.l_data, self.l_label = [], []
        for i in range(X.shape[0]):
            self.l_data.append(X[i, :, :, :])
            self.l_label.append(Y[i])

    def __getitem__(self, index):
        return self.l_data[index], self.l_label[index]

    def __len__(self):
        return len(self.l_data)

train_loader = paddle.io.DataLoader(MnistDataSet(X_train, Y_train),  batch_size=N_BATCH, shuffle=True)
test_loader = paddle.io.DataLoader(MnistDataSet(X_test, Y_test), batch_size=N_BATCH, shuffle=False)

(42000, 785)
(42000, 785)
(16000, 1, 28, 28)
(16000,)
(26000, 1, 28, 28)
(26000,)


# Build Model

In [4]:
class Net(paddle.nn.Layer):
    def __init__(self):
        super(Net, self).__init__()
        self.encoder = paddle.nn.Sequential(paddle.nn.Conv2D(1, 16, 3, 1),
                                            paddle.nn.MaxPool2D(2),
                                            paddle.nn.Flatten(1),
                                            paddle.nn.Linear(2704, 128),
                                            paddle.nn.ReLU(),
                                            paddle.nn.Linear(128, 10))

    def forward(self, x):
        out = self.encoder(x)
        return out

# Model Train and Model Save

In [5]:
print("model train")
model = paddle.Model(Net(), InputSpec([None, 1, 28, 28], 'float32', 'x'), InputSpec([None, 10], 'float32', 'x'))
model.prepare(paddle.optimizer.Adam(learning_rate=0.001, parameters=model.parameters()), paddle.nn.CrossEntropyLoss(), Accuracy())
model.fit(train_loader,
          test_loader,
          epochs=N_EPOCH,
          batch_size=N_BATCH,
          save_dir=S_PADDLE_MODEL_PATH + "_iter",
          verbose=1)
model.save(S_PADDLE_MODEL_PATH + "_final_model")
print()
# model.save(S_PADDLE_MODEL_PATH) # Model save


model train


RuntimeError: (PreconditionNotMet) The third-party dynamic library (cudnn64_7.dll) that Paddle depends on is not configured correctly. (error code is 126)
  Suggestions:
  1. Check if the third-party dynamic library (e.g. CUDA, CUDNN) is installed correctly and its version is matched with paddlepaddle you installed.
  2. Configure third-party dynamic library environment variables as follows:
  - Linux: set LD_LIBRARY_PATH by `export LD_LIBRARY_PATH=...`
  - Windows: set PATH by `set PATH=XXX; (at ..\paddle\fluid\platform\dynload\dynamic_loader.cc:285)
  [operator < gaussian_random > error]

# Model Predict

In [ ]:
print("model pred")
model.evaluate(test_loader, batch_size=N_BATCH, verbose=1)
print()

# Model Load and Loaded Model Use

In [ ]:
print("load model and pred test data")
model_load = paddle.Model(Net(), InputSpec([None, 1, 28, 28], 'float32', 'x'), InputSpec([None, 10], 'float32', 'x'))
# model_load.load(S_PADDLE_MODEL_PATH + "_iter/final")
model_load.load(S_PADDLE_MODEL_PATH + "_final_model")
model_load.prepare(paddle.optimizer.Adam(learning_rate=0.001, parameters=model.parameters()), paddle.nn.loss.CrossEntropyLoss(), Accuracy())
model_load.evaluate(test_loader, batch_size=N_BATCH, verbose=1)
print()

# Export Onnx

In [ ]:
x_spec = InputSpec([None, 1, 28, 28], 'float32', 'x')
paddle.onnx.export(Net(), S_ONNX_MODEL_PATH, input_spec=[x_spec])

# Onnx Run

In [ ]:
S_DEVICE = "cuda" if S_DEVICE == "gpu" else S_DEVICE
model = onnx.load(S_ONNX_MODEL_PATH + ".onnx")
print(onnx.checker.check_model(model))  # Check that the model is well formed
print(onnx.helper.printable_graph(model.graph))  # Print a human readable representation of the graph
ls_input_name, ls_output_name = [input.name for input in model.graph.input], [output.name for output in model.graph.output]
print("input name ", ls_input_name)
print("output name ", ls_output_name)
s_input_name = ls_input_name[0]

x_input = X_train[:N_BATCH * 2, :, :, :].astype(np.float32)
ort_val = ort.OrtValue.ortvalue_from_numpy(x_input, S_DEVICE, N_DEVICE_ID)
print("val device ", ort_val.device_name())
print("val shape ", ort_val.shape())
print("val data type ", ort_val.data_type())
print("is_tensor ", ort_val.is_tensor())
print("array_equal ", np.array_equal(ort_val.numpy(), x_input))
providers = 'CUDAExecutionProvider' if S_DEVICE == "cuda" else 'CPUExecutionProvider'
print("providers ", providers)
ort_session = ort.InferenceSession(S_ONNX_MODEL_PATH + ".onnx", providers=[providers])  # gpu运行
ort_session.set_providers([providers])
outputs = ort_session.run(None, {s_input_name: ort_val})
print("sess env ", ort_session.get_providers())
print(type(outputs))
print(outputs[0])
'''
For example ['CUDAExecutionProvider', 'CPUExecutionProvider']
    means execute a node using CUDAExecutionProvider if capable, otherwise execute using CPUExecutionProvider.
'''